In [ ]:
#@title #Runtime Info
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')


Sun Jun 23 23:22:02 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
#@title # Setup
%cd /content/
!apt install lshw
!curl -fsSL https://ollama.com/install.sh | sh
!git clone --depth 1 https://github.com/sam-paech/Ollama-MMLU-Pro-IRT.git
%cd Ollama-MMLU-Pro-IRT
!pip install -r requirements.txt


/content
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  pci.ids usb.ids
The following NEW packages will be installed:
  lshw pci.ids usb.ids
0 upgraded, 3 newly installed, 0 to remove and 35 not upgraded.
Need to get 791 kB of archives.
After this operation, 2,988 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 lshw amd64 02.19.git.2021.06.19.996aaad9c7-2build1 [321 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 pci.ids all 0.0~2022.01.22-1ubuntu0.1 [251 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 usb.ids all 2022.04.02-1 [219 kB]
Fetched 791 kB in 2s (388 kB/s)
Selecting previously unselected package lshw.
(Reading database ... 126308 files and directories currently installed.)
Preparing to unpack .../lshw_02.19.git.2021.06.19.996aaad9c7-2build1_amd64.deb ...
Unpacking lshw (02.19.git.2021.06.19.

In [ ]:
#@title # Run server

%env OLLAMA_NUM_PARALLEL=8
%env OLLAMA_FLASH_ATTENTION=1

import threading
import subprocess
server = threading.Thread(target=lambda: subprocess.run(['ollama', 'serve'], check=True), daemon=True)
server.start()
import  time

time.sleep(2)

In [ ]:
#@title Download
print("Downloading gemma3:4b")
!ollama pull hf.co/CALISTA-INDUSTRY/Gemma3_1B_Reasoning_En_FT_R16_LA16 &>/dev/null


In [ ]:
!pip install -U datasets

In [ ]:
#@title Run Test

!rm -rf eval_results
!python run_openai.py \
    --url http://localhost:11434/v1 \
    --model hf.co/CALISTA-INDUSTRY/Gemma3_1B_Reasoning_En_FT_R16_LA16 \
    --category 'computer science' \
    --parallel 2


README.md: 5.54kB [00:00, 19.4MB/s]
test-00000-of-00001.parquet: 100% 613k/613k [00:00<00:00, 9.68MB/s]
validation-00000-of-00001.parquet: 100% 45.3k/45.3k [00:00<00:00, 122MB/s]
Generating test split: 100% 2059/2059 [00:00<00:00, 31583.18 examples/s]
Generating validation split: 100% 70/70 [00:00<00:00, 8173.76 examples/s]
assigned subjects ['computer science']
Testing computer science...
100% 69/69 [15:40<00:00, 13.63s/it]

Correct: 8/69, Score: 11.59%

Total Correct: 8/69, Total Score: 11.59%
Finished the benchmark in 0 hours, 15 minutes, 43 seconds.
